# The NVSHMEM Library

## Introduction

Many scalable applications perform fine-grained communication that’s tightly coupled with computation. For such applications, existing communication libraries like MPI that support communication only at kernel boundaries can incur significant kernel launch. They place communication latencies on the critical path to performance. NVSHMEM is a communication library based on OpenSHMEM and designed specifically for clusters of NVIDIA GPUs. NVSHMEM can significantly reduce communication and coordination overheads by allowing programmers to perform these operations from within CUDA kernels and on CUDA streams. 

![nvshmem_memory_model](../../images/nvshmem_memory_model.png)

NVSHMEM provides a partitioned global address space (PGAS) that combines the memory of multiple GPUs into a shared address space that is accessed through the NVSHMEM API. The NVSHMEM API allows for fine-grained, remote data access that takes advantage of the massive parallelism in the GPU to hide communication overheads. By performing communication from within CUDA kernels, NVSHMEM allows us to write long running kernels that reduce the amount of overhead generated by synchronization with the CPU. As a result, NVSHMEM can significantly improve parallel efficiency.

**Note:** The NVSHMEM library requires GPUDirect P2P (for intra-node transfers) and GPUDirect RDMA (for inter-node transfers) technologies to be present.

## NVSHMEM compared with CUDA-aware MPI

The OpenSHMEM standard offers a partitioned global address space (PGAS) library for one-sided communication among CPU memories. NVSHMEM extends the standard OpenSHMEM APIs with support for communication from GPU threads, blocks, and warps;  direct data movement between GPU memories; and deferred execution APIs from the CPU that support CUDA Streams.

![nvshmem_mpi_comparison](../../images/nvshmem_mpi_comparison.png)

As shown above, MPI separates out computation and communication into different phases. Even though the data can be transferred directly from GPU memory as described earlier, the send and receive actions are performed on the CPU and serialization with kernel-based computation is necessary. \

* MPI implementations can incur high locking (or atomics) overheads for shared data structures that are involved in messaging.
* Serialization overheads that result from the MPI message ordering requirements.
* Protocol overheads that result from messages arriving at the receiver before they have posted the corresponding receive operation.

Current state-of-the-art applications that run on GPU clusters typically offload computation phases onto the GPU and rely on the CPU to manage communication between cluster nodes, by using MPI. Depending on the CPU for communication limits strong scalability because of the overhead of repeated kernel launches, CPU-GPU synchronization, underutilization of the GPU during communication phases, and underutilization of the network during compute phases. Some of these issues can be addressed by restructuring the application code to overlap independent compute and communication phases using CUDA streams. These optimizations can lead to complex application code and the benefits usually diminish as the problem size per GPU becomes smaller.

## GPU-initiated communication

NVSHMEM's memory model allows both computation and communication to occur from within the device kernel in the GPU. The benefits of this approach are as follows:

* It eliminates offload latencies to the CPU.
* It enables compute and communication overlap by hiding communication latency behind massive GPU parallelism.
* It makes it easier to express algorithms with inline communication.

NVSHMEM provides get and put APIs, which copy data from and to symmetric objects, respectively. Bulk transfer, scalar transfer, and interleaved versions of these APIs are provided. In addition, Atomic Memory Operations (AMOs) are also provided and can be used to perform atomic updates to symmetric variables. With these APIs, NVSHMEM provides fine-grained and low-overhead access to data that is stored in the PGAS from CUDA kernels. By performing communication from within the kernel, NVSHMEM also allows applications to benefit from the intrinsic latency-hiding capabilities of the GPU warp scheduling hardware.

NVSHMEM also allows any two CUDA threads in a job to synchronize on locations in global memory by using the OpenSHMEM point-to-point synchronization API `nvshmem_wait_until` or collective synchronization APIs like `nvshmem_barrier`.

### Communication Model

NVSHMEM provides get and put APIs, which copy data from and to symmetric objects, respectively. Bulk transfer, scalar transfer, and interleaved versions of these APIs are provided. In addition, Atomic Memory Operations (AMOs) are also provided and can be used to perform atomic updates to symmetric variables. With these APIs, NVSHMEM provides fine-grained and low-overhead access to data that is stored in the PGAS from CUDA kernels. By performing communication from within the kernel, NVSHMEM also allows applications to benefit from the intrinsic latency-hiding capabilities of the GPU warp scheduling hardware.

## Memory Model

An NVSHMEM program consists of data objects that are private to each PE and data objects that are remotely accessible by all PEs. Private data objects are stored in the local memory of each PE and can only be accessed by the PE itself; these data objects cannot be accessed by other PEs via NVSHMEM routines. Private data objects follow the memory model of C. Remotely accessible objects, however, can be accessed by remote PEs using NVSHMEM routines. Remotely accessible data objects are called Symmetric Data Objects. Each symmetric data object has a corresponding object with the same name, type, and size on all PEs where that object is accessible via the NVSHMEMAPI.

NVSHMEM dynamic memory allocation routines (e.g., nvshmem_malloc) allow collective allocation of Symmetric Data Objects on a special memory region called the Symmetric Heap. The Symmetric Heap is created during the execution of a program at a memory location determined by the NVSHMEM library. The Symmetric Heap may reside in different memory regions on different PEs. 

### Pointers to Symmetric Objects

Symmetric data objects are referenced in NVSHMEM operations through the local pointer to the desired remotely accessible object. The address contained in this pointer is referred to as a symmetric address. Every symmetric address is also a local address that is valid for direct memory access; however, not all local addresses are symmetric. Manipulation of symmetric addresses passed to NVSHMEM routines—including pointer arithmetic, array indexing, and access of structure or union members—are permitted as long as the resulting local pointer remains within the same symmetric allocation or object. Symmetric addresses are only valid at the PE where they were generated.

### Visibility Guarantees

On systems with both NVLink and InfiniBand, the NVSHMEM synchronization operations including nvshmem_barrier, nvshmem_barrier_all, nvshmem_quiet, nvshmem_wait_until_*, and nvshmem_test_* only guarantee visibility of updates to the local PE’s symmetric objects. However, on systems with only NVLink, these operations guarantee global visibility of updates to symmetric objects.

## Using NVSHMEM

An NVSHMEM job represents a single program, multiple data (SPMD) parallel execution. Each PE is assigned an integer identifier (ID), that ranges from zero to one less than the total number of PEs. PE IDs are used to identify the source or destination process in OpenSHMEM operations and are also used by application developers to assign work to specific processes in an NVSHMEM job.

All PEs in an NVSHMEM job must simultaneously, i.e. collectively, call the NVSHMEM initialization routine before an NVSHMEM operation can be performed. Similarly, before exiting, PEs must also collectively call the NVSHMEM finalization function. After initialization, a PE’s ID and the total number of running PEs can be queried. PEs communicate and share data through symmetric memory that is allocated from a symmetric heap that is located in GPU memory. This memory is allocated by using the CPU-side NVSHMEM allocation API. Memory that is allocated by using any other method is considered private to the allocating PE and is not accessible by other PEs.

### MPI and NVSHMEM

NVSHMEM can be used with MPI. The following code snippet shows how NVSHMEM can be initialized in an MPI program. In this program, we assume that each MPI process is also an NVSHMEM PE, where each process has both an MPI rank and an NVSHMEM rank.

```c
int main(int argc, char *argv[]) {
    int rank, ndevices;

    nvshmemx_init_attr_t attr;
    MPI_Comm comm = MPI_COMM_WORLD;
    attr.mpi_comm = &comm;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    cudaGetDeviceCount(&ndevices);
    cudaSetDevice(rank % ndevices);
    nvshmemx_init_attr(NVSHMEMX_INIT_WITH_MPI_COMM, &attr);

    // ...

    nvshmem_finalize();
    MPI_Finalize();
    return 0;
}
```

As shown in this example, the MPI library should be initialized first. After MPI is initialized, the MPI rank can be queried and used to set the CUDA device. An `nvshmemx_init_attr_t` structure is created and the `mpi_comm` field is assigned a reference to an MPI communicator handle. To enable MPI compatibility mode, the `nvshmemx_init_attr` operation is used instead of `nvshmem_init`. 

### Hello World Example: Left Shift Kernel with NVSHMEM and MPI

The code snippet below shows a simple example of NVSHMEM usage within a CUDA kernel where PEs form a communication ring.

```c
__global__ void simple_shift(int *destination) {
    int mype = nvshmem_my_pe();
    int npes = nvshmem_n_pes();
    int peer = (mype + 1) % npes;

    nvshmem_int_p(destination, mype, peer);
}

int main (int argc, char *argv[]) {
    int mype_node, msg;
    cudaStream_t stream;
    int rank, nranks;
    MPI_Comm mpi_comm = MPI_COMM_WORLD;
    nvshmemx_init_attr_t attr;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nranks);

    attr.mpi_comm = &mpi_comm;
    nvshmemx_init_attr(NVSHMEMX_INIT_WITH_MPI_COMM, &attr);
    mype_node = nvshmem_team_my_pe(NVSHMEMX_TEAM_NODE);

    cudaSetDevice(mype_node);
    cudaStreamCreate(&stream);
    int *destination = (int *) nvshmem_malloc (sizeof(int));

    simple_shift<<<1, 1, 0, stream>>>(destination);
    nvshmemx_barrier_all_on_stream(stream);
    cudaMemcpyAsync(&msg, destination, sizeof(int), cudaMemcpyDeviceToHost, stream);

    cudaStreamSynchronize(stream);
    printf("%d: received message %d\n", nvshmem_my_pe(), msg);

    nvshmem_free(destination);
    nvshmem_finalize();
    MPI_Finalize();
    return 0;
}
```

This example begins in main by initializing the NVSHMEM and MPI libraries, querying the PE’s ID in the on-node team, and using the on-node ID to set the CUDA device. The device must be set before you allocate memory or launch a kernel. A stream is created and a symmetric integer called `destination` is allocated on every PE. Finally, the `simple_shift` kernel is launched on one thread with a pointer to this symmetric object as its argument.

![nvshmem_left_shift_output](../../images/nvshmem_left_shift_output.png)

This kernel queries the global PE ID and the number of executing PEs. It then performs a single-element integer put operation to write the calling PE’s ID into destination at the PE with the next highest ID, or in the case of the PE with the highest ID, 0. The kernel is launched asynchronously on stream, followed by an NVSHMEM barrier on the stream to ensure that all updates have completed, and an asynchronous copy to copy the updated destination value to the host. The stream is synchronized and the result is printed. 

Finally, the destination buffer is freed and the NVSHMEM library is finalized before the program exits.

#### Compilation

Compiling NVSHMEM programs requires that the `NVSHMEM_HOME` environmental variable is set. A typical compilation command is given below:

```bash
nvcc -ccbin=mpic++ -gencode=$NVCC_GENCODE -I $NVSHMEM_HOME/include left_shift.cu -o left_shift -L $NVSHMEM_HOME/lib -lnvshmem -lcuda
```

Compile the Left Shift Hello World program:

In [ ]:
! cd ../../source_code/nvshmem/ && make clean && make left_shift

Now, run the program using `mpirun` command with 4 ranks and 1 rank per socket:

In [ ]:
! cd ../../source_code/nvshmem/ && mpirun -np 4 --map-by ppr:1:socket ./left_shift

Ensure that the output is consistent with the explanation provided above.

Now, let us learn more about NVSHMEM APIs.

### Heap Memory Allocation

```c
void *nvshmem_malloc(size_t size)
```

The `nvshmem_malloc` routine returns the symmetric address of a block of at least `size` bytes, which shall be suitably aligned so that it may be assigned to a pointer to any type of object. This space is allocated from the symmetric heap (in contrast to `malloc`, which allocates from the private heap). 

### Thread-level communication

To allow fine-grained thread-level communication and computation overlap, NVSHMEM provides thread-level communication APIs. 

![nvshmem_thread_level_comm](../../images/nvshmem_thread_level_comm.png)

The device kernel for a stencil-based program, like our Jacobi solver, looks as follows:

```c
__global__ void stencil_single_step(float *u, float *v, …) {
    int ix = get_ix(blockIdx, blockDim, threadIdx);
    int iy = get_iy(blockIdx, blockDim, threadIdx);
    compute(u, v, ix, iy);
    // Thread-level data communication API
    if (iy == 1)
        nvshmem_float_p(u+(ny+1)*nx+ix, u[nx+ix], top_pe);
    if (iy == ny)
        nvshmem_float_p(u+ix, u[ny*nx+ix], bottom_pe);
}
```

To ensure all communication is complete at the end of each iteration, use `nvshmem_barrier_all_on_stream` function in the host-side loop as follows:

```c
for (int iter = 0; iter < N; iter++) {
    swap(u, v);
    stencil_single_step<<<..., stream>>>(u, v, …);
    nvshmem_barrier_all_on_stream(stream);
}
```

### Thread-group level communication

NVSHMEM operations can be issued by all threads in a block/warp and this technique is more efficient for data transfers over networks like IB. It allows overlap at inter-warp or inter-block level.

Here's an example for block-level communication:

```c
// Thread block-level communication API
int boffset = get_block_offet(blockIdx,blockDim);
if (blockIdx.y == 0)
    nvshmemx_float_put_nbi_block(u+(ny+1)*nx+boffset, u+nx+boffset, blockDim.x, top_pe);
if (blockIdx.y == (blockDim.y-1))
    nvshmemx_float_put_nbi_block(u+boffset, u+ny*nx+boffset, blockDim.x, bottom_pe);
```

### Program-level Barrier

The `nvshmem_barrier_all` routine is a mechanism for synchronizing all PEs at once. This routine blocks the calling PE until all PEs have called `nvshmem_barrier_all`. In a multithreaded NVSHMEM program, only the calling thread is blocked, however, it may not be called concurrently by multiple threads in the same PE.

## Implementation Exercise

Open the [jacobi_nvshmem.cu](../../source_code/nvshmem/jacobi_nvshmem.cu) file. Alternatively, you can navigate to `CFD/English/C/source_code/mpi/` directory in Jupyter's file browser in the left pane. Then, click to open the `jacobi_nvshmem.cu` file.

Observe the following steps that occur prior to the Jacobi solver's iterative loop:

1. MPI is initialized, local rank and local size (number of GPUs per node) is determined, and current GPU is set. 
2. The default value of NVSHMEM's symmetric heap size is 1GB, which is insufficient for large grid sizes. This size is reset by setting the `NVSHMEM_SYMMETRIC_SIZE` environment variable from within the program.
3. NVSHMEM is initialized.
4. Current PE ID is stored in `mype` variable.
5. `nvshmem_malloc` allocates chunks of equal size across all PEs.

Now, NVSHMEM APIs like barriers work naturally with CUDA streams. Thus, we employ a small optimization called "delayed norm reset". We reset the norm in a separate `reset_l2_norm_stream` and perform norm device kernel computation, and device-local norm copy back to host on `compute_stream`.

We use `reset_l2_norm_done[curr]`event to ensure that norm reset is performed before we launch `jacobi_kernel` as we make the `compute_stream` wait on it. Moreover, we record the copy of the norm back to host in `l2_norm_bufs[curr].copy_done` event and synchronize this event before calling `MPI_AllReduce`. Finally, we reset everything before moving on to the next iteration and record the norm copy event in `reset_l2_norm_done[prev]`.

This ensures that the CPU-centric L2 norm reduction is kept separate in the GPU (using streams) from the GPU-centric Jacobi solver computations and GPU-initiated communications (within the device kernel).

After understanding the program flow, implement the following marked as `TODO`:

* In the `jacobi_kernel` device kernel:
  - Use block-level NVSHMEM put communication API to transfer the halo pointed to by `a_new + iy_start * nx + block_ix` to `a_new + top_iy * nx + block_ix` in the `top_pe`.
  - Similarily, transfer the halo pointed to by `a_new + (iy_end - 1) * nx + block_ix` to `a_new + bottom_iy * nx + block_ix` in the `bottom_pe`.
* In the iterative Jacobi (`while`) loop:
  - Use `cudaStreamWaitEvent` on `compute_stream` to wait for `reset_l2_norm_done` event to complete for `curr` iteration.
  - Put a barrier at the "compute_stream" stream level before performing L2 norming calculation.
  - Record the event `l2_norm_bufs[curr].copy_done` for `compute_stream`.
  - Record the L2 norm reset in `reset_l2_norm_done[prev]` for `reset_l2_norm_stream`. Understand why we use `prev` iteration: the stream waits for this event in the next iteration with `curr` iteration value.
  
After implementing these, compile the program:

In [ ]:
! cd ../../source_code/nvshmem/ && make clean && make jacobi_nvshmem

Ensure that there are no errors. Now, run the program with 16 GPUs across 2 nodes for $16K\times32K$ grid size:

In [ ]:
! cd ../../source_code/nvshmem/ && mpirun -np 16 --map-by ppr:4:socket ./jacobi_nvshmem -ny 32768

We share the partial output from our DGX-1V system:

```bash
Num GPUs: 16.
32768x16384: 1 GPU:   8.8382 s, 16 GPUs:   0.6216 s, speedup:    14.22, efficiency:    88.87 
```

Will we see a substancial improvement in performance by increasing number of iterations? Run the application for 5K iterations:

In [ ]:
! cd ../../source_code/nvshmem/ && mpirun -np 16 --map-by ppr:4:socket ./jacobi_nvshmem -ny 32768 -niter 5000

Our partial output is as follows:

```bash
Num GPUs: 16.
16384x32768: 1 GPU:  44.2000 s, 16 GPUs:   3.0736 s, speedup:    14.38, efficiency:    89.88
```

We don't see considerable gains in performance because NVSHMEM library aims to provide extremely low latency and therefore does not have high-overhead function calls.

Now, let us profile the application to learn more about how NVSHMEM is able to extract very high efficiency from multi-node runs of the application.

## Profiling

While profiling NVSHMEM-based programs, ensure that NVTX annotation-based instrumentation is enabled by setting the `NVSHMEM_NVTX` environment variable to `common`.

Profile the application using `nsys`. Skip the single-GPU run and use a $16K\times32K$ grid size.

In [ ]:
! cd ../../source_code/nvshmem && export NVSHMEM_NVTX=common && nsys profile \
    --trace=mpi,cuda,nvtx --stats=true --force-overwrite true -o jacobi_nvshmem_report \
                mpirun -np 16 --map-by ppr:4:socket ./jacobi_nvshmem -ny 32768 -skip_single_gpu

Download the report from [here](../../source_code/nvshmem/jacobi_nvshmem_report.dqrep).Notice that most NVSHMEM calls are available in NVTX Push-Pop stats on the CLI output. Open `.qdrep` file in the GUI to view the Timeline:

![nvshmem_profiler_report](../../images/nvshmem_profiler_report.png)

NVSHMEM is visible both on CUDA Hardware tab and on Threads tab. The total time between iterations has dropped to $45\mu$s and the `barrier_on_stream...` wrapper has a latency of about $35\mu$s. Thus, the idle time within each PE or GPU is $45-35=10\mu$s. Moreover, notice that the `MPI_Allreduce` function and HtoD copy of L2 norm back to host (denoted in green) happen concurrently with Jacobi device kernel run. 

This highly concurrent execution reduces GPU idle time and improves application performance. 

We now have an in-depth view of programming with NVSHMEM to achieve optimal performance by using GPU-initiated asynchronous communication and minimal synchronization calls. This concludes the overview of various multi-node multi-GPU programming models and approaches in this bootcamp.thethethethethe

Click link to the home notebook below through which all other notebooks are accessible:

# [HOME](../../../start_here.ipynb)

---
## Links and Resources

* [Concepts: Accelerating IO in data-centers](https://developer.nvidia.com/blog/accelerating-io-in-the-modern-data-center-network-io/)
* [Programming Concepts: NVSHMEM User Guide](https://docs.nvidia.com/hpc-sdk/nvshmem/api/docs/introduction.html)
* [Programming Concepts: Scaling Scientific Computing with NVSHMEM](https://developer.nvidia.com/blog/scaling-scientific-computing-with-nvshmem/)
* [Documentation: NCCL API](https://docs.nvidia.com/hpc-sdk/nvshmem/api/docs/api.html)
* [Code: Multi-GPU Programming Models](https://github.com/NVIDIA/multi-gpu-programming-models)
* [Code: GPU Bootcamp](https://github.com/gpuhackathons-org/gpubootcamp/)

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

## Licensing
Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.
